# Setup

Fall [these instructions](https://docs.astral.sh/uv/getting-started/installation/) to install `uv`. Then run the following command to set up your environment:

```bash
uv sync --quiet
```

In [1]:
# Imports
from robora import Workflow, SQLiteStorageProvider
from robora.sonar_query import SonarQueryHandler
from robora.mock_query import MockQueryHandler
from library.original_cyber_question import OriginalCyberModel, get_question_set
from data import COUNTRIES, DOMAINS
import pandas as pd

In [2]:
sqlstorage = SQLiteStorageProvider("original_cyber.db")
workflow = Workflow(
    SonarQueryHandler(OriginalCyberModel),
    sqlstorage,
    workers = 4,
    )
question_set = get_question_set(
    domains=['Energy','Foreign Affairs'],
    countries=COUNTRIES,
)
question_set.max_questions = None

Loaded 4 stored question hashes from original_cyber.db


In [3]:
await workflow.ask_multiple(question_set, return_results=False)

ask_multiple: gathering answers
ask_multiple_stream: starting for 386 questions with 4 workers
Processing question 1/386: Regarding the top-level state Organ (i.e., ministry/department/agency) of {domain} in {country}: when did the Organ become responsible for cybersecurity? If applicable, answer in the following schema:  - {'domain': 'ENERGY', 'country': 'BOLIVIA'}
Processing question 2/386: Regarding the top-level state Organ (i.e., ministry/department/agency) of {domain} in {country}: when did the Organ become responsible for cybersecurity? If applicable, answer in the following schema:  - {'domain': 'ENERGY', 'country': 'NETHERLANDS'}
Processing question 3/386: Regarding the top-level state Organ (i.e., ministry/department/agency) of {domain} in {country}: when did the Organ become responsible for cybersecurity? If applicable, answer in the following schema:  - {'domain': 'FOREIGN AFFAIRS', 'country': 'CANADA'}
Processing question 4/386: Regarding the top-level state Organ (i.e., m

[]

In [4]:
answers = []
async for answer in workflow.dump_answers():
    answers.append(answer)
flattened = pd.concat([ans.flattened for ans in answers]).reset_index(drop=True)
flattened

,question,error,domain,country,involved,earliest_date,incarnation,confidence,explanation,enriched_citations
0,"Regarding the top-level state Organ (i.e., min...",None,ENERGY,TURKMENISTAN,True,2024-01-30,Ministry of Energy (or associated agency withi...,ConfidenceLevel.MEDIUM,Based on strategic cybersecurity indicators fo...,"[{'url': 'https://ncsi.ega.ee/country/tm/', 't..."
1,"Regarding the top-level state Organ (i.e., min...",None,ENERGY,ICELAND,True,2013-06-01,Ministry of the Interior,ConfidenceLevel.HIGH,A task force appointed by the Ministry of the ...,[{'url': 'https://www.stjornarradid.is/library...
2,"Regarding the top-level state Organ (i.e., min...",None,FOREIGN AFFAIRS,KAZAKHSTAN,False,None,None,ConfidenceLevel.HIGH,The Ministry of Foreign Affairs of Kazakhstan ...,[{'url': 'https://jamestown.org/program/kazakh...
3,"Regarding the top-level state Organ (i.e., min...",None,FOREIGN AFFAIRS,VIETNAM,True,2018-06-01,Ministry of Foreign Affairs of Vietnam,ConfidenceLevel.HIGH,The Ministry of Foreign Affairs of Vietnam is ...,[{'url': 'https://www.economica.vn/Content/fil...
4,"Regarding the top-level state Organ (i.e., min...",None,FOREIGN AFFAIRS,CANADA,True,2004-01-01,Global Affairs Canada (GAC),ConfidenceLevel.HIGH,Global Affairs Canada has engaged in cyber for...,[{'url': 'https://www.justsecurity.org/71817/c...
...,...,...,...,...,...,...,...,...,...,...
381,"Regarding the top-level state Organ (i.e., min...",None,ENERGY,NEPAL,False,None,None,ConfidenceLevel.HIGH,Available search results and policy documents ...,[{'url': 'https://www.naseo.org/data/sites/1/d...
382,"Regarding the top-level state Organ (i.e., min...",None,FOREIGN AFFAIRS,CAMEROON,False,None,None,ConfidenceLevel.HIGH,Cybersecurity in Cameroon is principally overs...,[{'url': 'https://www.ictacam.org/overview-of-...
383,"Regarding the top-level state Organ (i.e., min...",None,ENERGY,BHUTAN,True,2016-07-01,Bhutan Electricity Authority (BEA),ConfidenceLevel.MEDIUM,"The Bhutan Electricity Authority, the top-leve...",[{'url': 'https://www.bicma.gov.bt/data/report...
384,"Regarding the top-level state Organ (i.e., min...",None,FOREIGN AFFAIRS,PANAMA,False,None,None,ConfidenceLevel.HIGH,The responsibility for national cybersecurity ...,[{'url': 'https://dig.watch/resource/panamas-n...


In [5]:
wrangled = flattened.copy()


In [8]:
wrangled.enriched_citations

0      [{'url': 'https://ncsi.ega.ee/country/tm/', 't...
1      [{'url': 'https://www.stjornarradid.is/library...
2      [{'url': 'https://jamestown.org/program/kazakh...
3      [{'url': 'https://www.economica.vn/Content/fil...
4      [{'url': 'https://www.justsecurity.org/71817/c...
                             ...                        
381    [{'url': 'https://www.naseo.org/data/sites/1/d...
382    [{'url': 'https://www.ictacam.org/overview-of-...
383    [{'url': 'https://www.bicma.gov.bt/data/report...
384    [{'url': 'https://dig.watch/resource/panamas-n...
385    [{'url': 'https://pitiviti.org/storage/dm/2022...
Name: enriched_citations, Length: 386, dtype: object

In [7]:
wrangled.to_excel("original_cyber_answers.xlsx", index=False)